In [38]:
import torch

In [66]:

def euler_to_matrix(vec_rot):
    """Converts Euler angles to rotation matrix
    Args:
        vec_rot: Euler angles in the order of rx, ry, rz -- [B, 3] torch.tensor
    Returns:
        A rotation matrix -- [B, 3, 3] torch.tensor
    """
    batch_size = vec_rot.shape[0]
    rx, ry, rz = vec_rot[:, 0], vec_rot[:, 1], vec_rot[:, 2]
    
    cos_rx, sin_rx = torch.cos(rx), torch.sin(rx)
    cos_ry, sin_ry = torch.cos(ry), torch.sin(ry)
    cos_rz, sin_rz = torch.cos(rz), torch.sin(rz)
    
    R_x = torch.stack([torch.ones(batch_size), torch.zeros(batch_size), torch.zeros(batch_size),
                       torch.zeros(batch_size), cos_rx, -sin_rx,
                       torch.zeros(batch_size), sin_rx, cos_rx], dim=1).view(batch_size, 3, 3)
    
    R_y = torch.stack([cos_ry, torch.zeros(batch_size), sin_ry,
                       torch.zeros(batch_size), torch.ones(batch_size), torch.zeros(batch_size),
                       -sin_ry, torch.zeros(batch_size), cos_ry], dim=1).view(batch_size, 3, 3)
    
    R_z = torch.stack([cos_rz, -sin_rz, torch.zeros(batch_size),
                       sin_rz, cos_rz, torch.zeros(batch_size),
                       torch.zeros(batch_size), torch.zeros(batch_size), torch.ones(batch_size)], dim=1).view(batch_size, 3, 3)
    
    rotation_matrix = torch.bmm(R_z, torch.bmm(R_y, R_x))
    
    return rotation_matrix

def dof_vec_to_matrix(dof_vec):
    """Converts 6DoF parameters to transformation matrix
    Args:
        vec: 6DoF parameters in the order of tx, ty, tz, rx, ry, rz -- [B, 6] torch.tensor
    Returns:
        A transformation matrix -- [B, 4, 4] torch.tensor
        R11 R12 R13 tx
        R21 R22 R23 ty
        R31 R32 R33 tz
        0   0   0   1
    """
    batch_size = dof_vec.shape[0]
    translation = dof_vec[:,:3]
    # Add a one at the end of translation
    ones = torch.ones(batch_size, 1)
    translation = torch.cat((translation, ones), dim=1)
    rot_vec = dof_vec[:, 3:]
    # print("rot_vec", rot_vec)
    rot_matrix = euler_to_matrix(rot_vec)
    # add zero at 4 row
    zeros = torch.zeros(batch_size, 1, 3)
    rot_matrix = torch.cat((rot_matrix, zeros), dim=1)
    transformation_matrix = torch.cat((rot_matrix, translation.unsqueeze(2)), dim=2)
    return transformation_matrix

def inverse_dof(dof_vec):
    """
    Computes the inverse of 6DoF parameters.
    
    Args:
        dof_vec: Tensor of shape [B, 6], representing 6DoF parameters (tx, ty, tz, rx, ry, rz).
    
    Returns:
        Inverted 6DoF parameters: Tensor of shape [B, 6].
    """
    # Negate both the translation and rotation parts
    translation_inv = -dof_vec[:, :3]
    rotation_inv = -dof_vec[:, 3:]
    return torch.cat((translation_inv, rotation_inv), dim=1)


In [ ]:
def step_cloud(I_t, dof_vec):
    """
    Applies a 6DoF transformation to a point cloud.
    
    Args:
        I_t: Tensor of shape [B, N, 3], representing a batch of point clouds.
        dof_vec: Tensor of shape [B, 6], representing 6DoF parameters (tx, ty, tz, rx, ry, rz).
    
    Returns:
        I_t_1: Transformed point cloud, Tensor of shape [B, N, 3].
    """
    batch_size, num_points = I_t.shape[0], I_t.shape[1]
    
    # Step 1: Convert to homogeneous coordinates
    ones = torch.ones(batch_size, num_points, 1, device=I_t.device)  # [B, N, 1]
    I_t_augmented = torch.cat((I_t, ones), dim=2)  # [B, N, 4]
    
    # Step 2: Get the transformation matrix
    transf_mat = dof_vec_to_matrix(dof_vec)  # [B, 4, 4]
    
    # Step 3: Apply the transformation
    # Transpose the transformation matrix for compatibility
    transf_mat = transf_mat.transpose(1, 2)  # [B, 4, 4]
    I_t_1_homo = torch.bmm(I_t_augmented, transf_mat)  # [B, N, 4]
    
    # Step 4: Convert back to Cartesian coordinates
    I_t_1 = I_t_1_homo[:, :, :3]  # Drop the homogeneous coordinate
    
    return I_t_1

def photo_Loss(I, I_pred):
    pass
    #return photo_loss

In [74]:
def pixel_to_3d(points, intrins):
    """
    Converts pixel coordinates and depth to 3D coordinates.
    
    Args:
        points: Tensor of shape [B, N, 3], where B is the batch size, N is the number of points,
                and each point is represented as (u, v, w) where u and v are pixel coordinates and w is depth.
        intrins: List of intrinsic parameters of the camera.
    
    Returns:
        Tensor of shape [B, N, 3] representing the 3D coordinates.
    """
    fx = intrins[0][0]
    fy = intrins[1][1]
    cx = intrins[0][2]
    cy = intrins[1][2]
    
    u = points[:, :, 0]
    v = points[:, :, 1]
    w = points[:, :, 2]
    
    x = ((u - cx) * w) / fx
    y = ((v - cy) * w) / fy
    z = w
    
    return torch.stack((x, y, z), dim=2)

# Example usage
points_batch = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
intrins_example = [
    [956.9475, 0.0, 693.9767],
    [0.0, 952.2352, 238.6081],
    [0.0, 0.0, 1.0]
]

points_3d = pixel_to_3d(points_batch, intrins_example)
print(points_3d)

tensor([[[-2.1725, -0.7454,  3.0000],
         [-4.3261, -1.4720,  6.0000]],

        [[-6.4609, -2.1796,  9.0000],
         [-8.5770, -2.8683, 12.0000]]])


In [76]:
def _3d_to_pixel(points_3d, intrins):
    """
    Converts 3D coordinates to pixel coordinates and depth.
    
    Args:
        points_3d: Tensor of shape [B, N, 3], where B is the batch size, N is the number of points,
                   and each point is represented as (x, y, z) where x, y, and z are 3D coordinates.
        intrins: List of intrinsic parameters of the camera.
    
    Returns:
        Tensor of shape [B, N, 3] representing the pixel coordinates and depth.
    """
    fx = intrins[0][0]
    fy = intrins[1][1]
    cx = intrins[0][2]
    cy = intrins[1][2]
    
    x = points_3d[:, :, 0]
    y = points_3d[:, :, 1]
    z = points_3d[:, :, 2]
    
    u = (x * fx / z) + cx
    v = (y * fy / z) + cy
    w = z
    
    return torch.stack((u, v, w), dim=2)

# Example usage
points_3d_example = points_3d
intrins_example = [
    [956.9475, 0.0, 693.9767],
    [0.0, 952.2352, 238.6081],
    [0.0, 0.0, 1.0]
]

pixels = _3d_to_pixel(points_3d_example, intrins_example)
print(pixels)

tensor([[[ 0.9999,  2.0000,  3.0000],
         [ 3.9999,  5.0000,  6.0000]],

        [[ 7.0000,  8.0000,  9.0000],
         [ 9.9999, 11.0000, 12.0000]]])


In [ ]:
# unit test dimensions
torch.manual_seed(42)
batch = 1
num_points = 2
I_t_example = torch.randint(0, 10, (batch, num_points, 3))
dof_vec_example = torch.randint(0, 10, (batch, 6))

print('I_t_example\n', I_t_example)
print('I_t_example.shape', I_t_example.shape)
print("dof_vec_example\n", dof_vec_example)
print('dof_vec_example.shape', dof_vec_example.shape)

step_cloud(I_t_example, dof_vec_example)

I_t_example
 tensor([[[2, 7, 6],
         [4, 6, 5]]])
I_t_example.shape torch.Size([1, 2, 3])
dof_vec_example
 tensor([[0, 4, 0, 3, 8, 4]])
dof_vec_example.shape torch.Size([1, 6])


tensor([[[-2.4927, 13.0113, -1.2582],
         [-1.9954, 11.8566, -3.3604]]])

In [69]:
# unit test, invertible?
# Example Input
I_t = torch.rand(2, 5, 3)  # Original Point Cloud
dof_vec = torch.tensor([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]], dtype=torch.float32)  # Translation only

# Step 1: Transform the point cloud
transformed_cloud = step_cloud(I_t, dof_vec)

# Step 2: Compute the inverse 6DoF
inverse_dof_vec = inverse_dof(dof_vec)

# Step 3: Apply the inverse transformation
recovered_cloud = step_cloud(transformed_cloud, inverse_dof_vec)

# Check if input matches recovered cloud
print("Original Point Cloud:\n", I_t)
print("Transformed Point Cloud:\n", transformed_cloud)
print("Recovered Point Cloud:\n", recovered_cloud)

assert torch.allclose(I_t, recovered_cloud, atol=1e-6), "The original and recovered point clouds do not match!"

Original Point Cloud:
 tensor([[[0.4340, 0.1371, 0.5117],
         [0.1585, 0.0758, 0.2247],
         [0.0624, 0.1816, 0.9998],
         [0.5944, 0.6541, 0.0337],
         [0.1716, 0.3336, 0.5782]],

        [[0.0600, 0.2846, 0.2007],
         [0.5014, 0.3139, 0.4654],
         [0.1612, 0.1568, 0.2083],
         [0.3289, 0.1054, 0.9192],
         [0.4008, 0.9302, 0.6558]]])
Transformed Point Cloud:
 tensor([[[1.4340, 0.1371, 0.5117],
         [1.1585, 0.0758, 0.2247],
         [1.0624, 0.1816, 0.9998],
         [1.5944, 0.6541, 0.0337],
         [1.1716, 0.3336, 0.5782]],

        [[0.0600, 1.2846, 0.2007],
         [0.5014, 1.3139, 0.4654],
         [0.1612, 1.1568, 0.2083],
         [0.3289, 1.1054, 0.9192],
         [0.4008, 1.9302, 0.6558]]])
Recovered Point Cloud:
 tensor([[[0.4340, 0.1371, 0.5117],
         [0.1585, 0.0758, 0.2247],
         [0.0624, 0.1816, 0.9998],
         [0.5944, 0.6541, 0.0337],
         [0.1716, 0.3336, 0.5782]],

        [[0.0600, 0.2846, 0.2007],
       

In [ ]:
batch = 2

example_input = torch.randint(0, 10, (batch, 6))

print(example_input)
print(dof_vec_to_matrix(example_input))

tensor([[2, 7, 6, 4, 6, 5],
        [0, 4, 0, 3, 8, 4]])
tensor([[[ 0.2724, -0.5668,  0.7775,  2.0000],
         [-0.9207, -0.3882,  0.0395,  7.0000],
         [ 0.2794, -0.7267, -0.6276,  6.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.0951, -0.8405,  0.5334,  0.0000],
         [ 0.1101,  0.5414,  0.8335,  4.0000],
         [-0.9894, -0.0205,  0.1440,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])
